## Stock Portfolio Optimisation through Sharpe Ratio Using Dispersive Flies Optimisation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

from utils.data_loader import get_stock_returns


 Data: daily percentage returns of top 120 S&P500 companies by market cap (2015-01-06 to 2025-11-04), excluding columns (tickers) with missing/no data (['BRK.B', 'PLTR', 'APP', 'UBER', 'GEV', 'CRWD', 'HOOD', 'CEG', 'DELL']) and rows (days) with missing/no data (e.g. no-trading days, null)

Sharpe = (expected return of portfolio – risk-free rate) / portfolio volatility